<a href="https://colab.research.google.com/github/AshvinVignesh/DL_CW_2/blob/main/Bert_Torch_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.0 MB/s eta 0:00:00


In [2]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__


('4.35.2', '0.24.1')

In [3]:
import pandas as pd

In [5]:
cleaned = pd.read_csv("yelp_reviews_2500.csv")
cleaned=cleaned.loc[:, ['stars', 'text']]
cleaned


,stars,text
0,3.0,Cool structure to go see if visiting Nashville...
1,3.0,We planned a trip to Philly around my husbands...
2,3.0,Always good service. Room was clean except the...
3,3.0,"This place has been here for years and years, ..."
4,3.0,This is a very large location (with a ton of T...
...,...,...
2495,2.0,10/2/16 Update. I have been back twice since ...
2496,2.0,We were excited about Certa Pro completing our...
2497,2.0,Driven by numerous times and finally came in. ...
2498,2.0,Let me say that the service was excellent. The...


In [6]:
# cleaned=cleaned.drop(cleaned.columns[[0,1,2,4,5,6,8]], axis = 1)
# cleaned

In [7]:
cleaned['stars'].value_counts()

3.0    500
5.0    500
4.0    500
1.0    500
2.0    500
Name: stars, dtype: int64

In [8]:
def map_sentiment_scores(star_number):
    star_number = int(star_number)
    if star_number == 1:
        return 0
    elif star_number == 2:
        return 1
    elif star_number == 3:
        return 2
    elif star_number == 4:
        return 3
    elif star_number == 5:
        return 4

In [9]:
cleaned['stars'] = cleaned.stars.apply(map_sentiment_scores)
cleaned.head(5)

,stars,text
0,2,Cool structure to go see if visiting Nashville...
1,2,We planned a trip to Philly around my husbands...
2,2,Always good service. Room was clean except the...
3,2,"This place has been here for years and years, ..."
4,2,This is a very large location (with a ton of T...


In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [11]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=5)

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
model = model.to('cuda')

In [13]:
sample_data = ["I am eating","I am playing "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [14]:
X = list(cleaned['text'])
y = list(cleaned['stars'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [15]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [16]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [17]:
len(X_train),len(X_val)

(2000, 500)

In [18]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [19]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [20]:
train_dataset[5]

{'input_ids': tensor([  101,  1045,  2074,  4156,  1996,  6265,  3482, 11642,  1998,  2009,
          2001, 11937, 21756,  1012,  2174,  1010,  2045,  2001,  4510,  2151,
          5624, 14540, 10376,  2030,  2845, 11225,  2006,  1996, 11642,  1998,
          2005,  1002,  2321,  1012,  4002,  2008,  1005,  1055,  2025,  7929,
          1012,  2045,  2024,  1017,  6240,  2015,  2006,  2023, 11642,  1998,
          2009,  2323,  2031,  2042, 21305,  2012,  1996, 25180,  2015,  2009,
          2347,  1005,  1056,  1012,  2009,  2134,  1005,  1056, 27304,  1010,
         13950,  2030,  2031,  1037,  4060,  2571,  2006,  1996,  2217,  1012,
          3531,  2202,  2023,  2592,  1998,  5335,  1012,  2017,  1005,  2128,
          4851,  2005,  1002,  2321,  1012,  4002,   999,   999,  2763,  2026,
          2034,  1998,  2197,  2051,  1012, 15488,  2232,   999,   999,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [21]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred,average='micro')
    precision = precision_score(y_true=labels, y_pred=pred,average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred,average='micro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [22]:

# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps = 50,
    eval_steps = 50,
    evaluation_strategy="steps",


)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


In [23]:
import matplotlib.pyplot as plt

In [24]:
train = trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,No log,1.350556,0.466000,0.466000,0.466000,0.466000
100,No log,1.078980,0.512000,0.512000,0.512000,0.512000
150,No log,1.033867,0.524000,0.524000,0.524000,0.524000
200,No log,1.035496,0.560000,0.560000,0.560000,0.560000
250,No log,1.023311,0.540000,0.540000,0.540000,0.540000
300,No log,1.133129,0.504000,0.504000,0.504000,0.504000
350,No log,1.173834,0.546000,0.546000,0.546000,0.546000
400,No log,1.064499,0.558000,0.558000,0.558000,0.558000
450,No log,1.085363,0.562000,0.562000,0.562000,0.562000
500,0.939700,1.050413,0.556000,0.556000,0.556000,0.556000


<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>


In [31]:
train_metrics = trainer.state.log_history
train_metrics
# # Extract relevant information
# train_loss = [metric['train_loss'] for metric in train_metrics]  # Training loss
# train_acc = [metric['eval_accuracy'] for metric in train_metrics]  # Training accuracy, if available

[{'eval_loss': 1.350555658340454,
  'eval_accuracy': 0.466,
  'eval_precision': 0.466,
  'eval_recall': 0.466,
  'eval_f1': 0.466,
  'eval_runtime': 15.8253,
  'eval_samples_per_second': 31.595,
  'eval_steps_per_second': 3.981,
  'epoch': 0.2,
  'step': 50},
 {'eval_loss': 1.0789796113967896,
  'eval_accuracy': 0.512,
  'eval_precision': 0.512,
  'eval_recall': 0.512,
  'eval_f1': 0.512,
  'eval_runtime': 15.6033,
  'eval_samples_per_second': 32.045,
  'eval_steps_per_second': 4.038,
  'epoch': 0.4,
  'step': 100},
 {'eval_loss': 1.0338666439056396,
  'eval_accuracy': 0.524,
  'eval_precision': 0.524,
  'eval_recall': 0.524,
  'eval_f1': 0.524,
  'eval_runtime': 15.736,
  'eval_samples_per_second': 31.774,
  'eval_steps_per_second': 4.004,
  'epoch': 0.6,
  'step': 150},
 {'eval_loss': 1.035495638847351,
  'eval_accuracy': 0.56,
  'eval_precision': 0.56,
  'eval_recall': 0.56,
  'eval_f1': 0.56,
  'eval_runtime': 15.7316,
  'eval_samples_per_second': 31.783,
  'eval_steps_per_second':

In [32]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 1.2394604682922363,
 'eval_accuracy': 0.568,
 'eval_precision': 0.568,
 'eval_recall': 0.568,
 'eval_f1': 0.568,
 'eval_runtime': 15.7046,
 'eval_samples_per_second': 31.838,
 'eval_steps_per_second': 4.012,
 'epoch': 3.0}

In [27]:
evaluate_metrics = trainer.state.log_history
evaluate_metrics

[{'eval_loss': 1.350555658340454,
  'eval_accuracy': 0.466,
  'eval_precision': 0.466,
  'eval_recall': 0.466,
  'eval_f1': 0.466,
  'eval_runtime': 15.8253,
  'eval_samples_per_second': 31.595,
  'eval_steps_per_second': 3.981,
  'epoch': 0.2,
  'step': 50},
 {'eval_loss': 1.0789796113967896,
  'eval_accuracy': 0.512,
  'eval_precision': 0.512,
  'eval_recall': 0.512,
  'eval_f1': 0.512,
  'eval_runtime': 15.6033,
  'eval_samples_per_second': 32.045,
  'eval_steps_per_second': 4.038,
  'epoch': 0.4,
  'step': 100},
 {'eval_loss': 1.0338666439056396,
  'eval_accuracy': 0.524,
  'eval_precision': 0.524,
  'eval_recall': 0.524,
  'eval_f1': 0.524,
  'eval_runtime': 15.736,
  'eval_samples_per_second': 31.774,
  'eval_steps_per_second': 4.004,
  'epoch': 0.6,
  'step': 150},
 {'eval_loss': 1.035495638847351,
  'eval_accuracy': 0.56,
  'eval_precision': 0.56,
  'eval_recall': 0.56,
  'eval_f1': 0.56,
  'eval_runtime': 15.7316,
  'eval_samples_per_second': 31.783,
  'eval_steps_per_second':

In [33]:
text = """Decent poke at an okay price. We got two bowls with sushi rice: salmon and albacore, and salmon and seared cajun tuna.
Good: The salmon and tuna were good, and there's a large selection of  mixins and addons -- I consistently get the crab salad, seaweed salad, and tofu skins.

Bad: The albacore was overly salty and many of the pieces were overcooked and rubbery. The sauces are good but I wish there was more added, without having to pay extra.

Overall, nothing spectacular, but Pokeman is an average option in University City."""
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[-2.1575,  0.2992,  3.9865,  1.1018, -1.9353]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.0020, 0.0231, 0.9210, 0.0515, 0.0025]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)


array([[0.00197685, 0.02306304, 0.92102945, 0.05146185, 0.00246887]],
      dtype=float32)

In [34]:
print("Ratings :" ,float(np.argmax(predictions) + 1 ))

Ratings : 3.0


In [30]:
torch.save(model, '/content/fine_tuned') #to save

saved_model = torch.load('/content/fine_tuned') # to load